In [1]:
import xgboost as xgb

## SAVIC-Q - C

### load SAVIC-Q - C XGB regressor

In [2]:
xgbr_c1_c = xgb.XGBRegressor()
xgbr_c1_c.load_model('Output/ML/models/xgbr_c.json')

### load SAVIC-Q - C processing function

In [3]:
def SAVIC_Q_C(df_c_):
    
    df_c_uns_ = df_c_[df_c_['unstable']].drop(columns = ['unstable'])
    
    df_c_uns_pre_SQ_ = df_c_uns_.copy()
    df_c_uns_pre_SQ_ = np.log10(df_c_uns_pre_SQ_)
    df_c_uns_pre_SQ_.columns = [['log_beta_par_core', 'log_alph_c']]
    
    df_c_uns_pre_SQ_[['log_Pc', 'log_theta_kB']] = xgbr_c1_c.predict(df_c_uns_pre_SQ_.values)
    
    df_c_uns_post_SQ_ = pow(10., df_c_uns_pre_SQ_)
    df_c_uns_post_SQ_.columns = ['beta_par_core', 'alph_c', 'Pc', 'theta_kB']
    
    df_c_post_SQ_ = df_c_.copy()
    df_c_post_SQ_[['Pc', 'theta_kB']] = np.nan
    df_c_post_SQ_.loc[df_c_uns_post_SQ_.index, ['Pc', 'theta_kB']] = df_c_uns_post_SQ_[['Pc', 'theta_kB']]
    
    return df_c_post_SQ_
    

### run SAVIC-Q - C 

In [4]:
df_c_post_SQ = SAVIC_Q_C(pd.read_hdf('00_SAVIC_Examples/SAVIC_Examples.h5', key = 'SAVIC-P_C_post_P'))
df_c_post_SQ

,beta_par_core,alph_c,unstable,Pc,theta_kB
0,1.0,1.0,False,NaN,NaN
1,0.5,3.2,True,0.171887,0.001113
2,1.0,0.4,True,0.000287,0.002231
3,12.0,1.2,True,0.001800,0.001649


## SAVIC-Q - CB

### load SAVIC-Q - CB XGB classifier \& regressor

In [5]:
xgbc_kcb = xgb.XGBClassifier()
xgbc_kcb.load_model('Output/ML/models/xgbc_kcb.json')

xgbr_c1_b1 = xgb.XGBRegressor()
xgbr_c1_b1.load_model('Output/ML/models/xgbr_cb_c1_b1.json')

xgbr_c1_b0 = xgb.XGBRegressor()
xgbr_c1_b0.load_model('Output/ML/models/xgbr_cb_c1_b0.json')

xgbr_c0_b1_k0 = xgb.XGBRegressor()
xgbr_c0_b1_k0.load_model('Output/ML/models/xgbr_cb_c0_b1_k0.json')

xgbr_c0_b1_k1 = xgb.XGBRegressor()
xgbr_c0_b1_k1.load_model('Output/ML/models/xgbr_cb_c0_b1_k1.json')

### load SAVIC-Q - CB processing function

In [6]:
def SAVIC_Q_CB(df_cb_):
    
    df_cb_uns_ = df_cb_[df_cb_['unstable']].drop(columns = ['unstable'])
    df_cb_uns_pre_SQ_ = df_cb_uns_.copy()
    df_cb_uns_pre_SQ_[['beta_par_core', 'alph_c', 'alph_b']] = \
    np.log10(df_cb_uns_pre_SQ_[['beta_par_core', 'alph_c', 'alph_b']])
    df_cb_uns_pre_SQ_.columns = [['log_beta_par_core', 'log_alph_c', 'tau_b', 'log_alph_b', 'D_b', 'vv_b']]
    
    df_cb_uns_postC_SQ_ = df_cb_uns_pre_SQ_.copy()
    df_cb_uns_postC_SQ_['group'] = xgbc_kcb.predict(df_cb_uns_pre_SQ_.values)
    
    df_cb_uns_preQ_SQ_ = df_cb_uns_postC_SQ_.copy()
    df_cb_uns_preQ_SQ_[['log_beta_par_core', 'log_alph_c', 'log_alph_b']] = \
    pow(10., df_cb_uns_preQ_SQ_[['log_beta_par_core', 'log_alph_c', 'log_alph_b']]) 
    df_cb_uns_preQ_SQ_.columns = ['beta_par_core', 'alph_c', 'tau_b', 'alph_b', 'D_b', 'vv_b', 'group']
    
    # regressors
    df_cb_uns_preQ_SQ_0_ = df_cb_uns_preQ_SQ_[df_cb_uns_preQ_SQ_['group'] == 0].drop(columns = ['group'])
    df_cb_uns_preQ_SQ_1_ = df_cb_uns_preQ_SQ_[df_cb_uns_preQ_SQ_['group'] == 1].drop(columns = ['group'])
    df_cb_uns_preQ_SQ_2_ = df_cb_uns_preQ_SQ_[df_cb_uns_preQ_SQ_['group'] == 2].drop(columns = ['group'])
    df_cb_uns_preQ_SQ_3_ = df_cb_uns_preQ_SQ_[df_cb_uns_preQ_SQ_['group'] == 3].drop(columns = ['group'])
    df_cb_uns_preQ_SQ_4_ = df_cb_uns_preQ_SQ_[df_cb_uns_preQ_SQ_['group'] == 4].drop(columns = ['group'])
    df_cb_uns_preQ_SQ_5_ = df_cb_uns_preQ_SQ_[df_cb_uns_preQ_SQ_['group'] == 5].drop(columns = ['group'])
    
    
    if df_cb_uns_preQ_SQ_0_.size > 0:
        df_cb_uns_preQ_SQ_0_log_ = df_cb_uns_preQ_SQ_0_.copy()

        df_cb_uns_preQ_SQ_0_log_[['beta_par_core', 'alph_c','alph_b']] = \
        np.log10(df_cb_uns_preQ_SQ_0_log_[['beta_par_core', 'alph_c','alph_b']])
        
        df_cb_uns_preQ_SQ_0_log_[['Pow_core', 'Pow_beam', 'kB_angle']] = \
        xgbr_c1_b1.predict(df_cb_uns_preQ_SQ_0_log_).astype(float)

        df_cb_uns_preQ_SQ_0_log_[['beta_par_core', 'alph_c','alph_b', 'Pow_core', 'Pow_beam', 'kB_angle']] = \
        10**df_cb_uns_preQ_SQ_0_log_[['beta_par_core', 'alph_c','alph_b', 'Pow_core', 'Pow_beam', 'kB_angle']]

        df_cb_uns_postQ_SQ_0_ = df_cb_uns_preQ_SQ_0_log_.copy()
        df_cb_uns_postQ_SQ_0_ = df_cb_uns_postQ_SQ_0_[['beta_par_core', 'alph_c', 'tau_b', 'D_b', 'alph_b', 'vv_b', 'Pow_core', 'Pow_beam', 'kB_angle']]
    else:
        df_cb_uns_postQ_SQ_0_ = pd.DataFrame(columns = ['beta_par_core', 'alph_c', 'tau_b', 'D_b', 'alph_b', 'vv_b', 'Pow_core', 'Pow_beam', 'kB_angle'])

    if df_cb_uns_preQ_SQ_1_.size > 0:
        df_cb_uns_preQ_SQ_1_log_ = df_cb_uns_preQ_SQ_1_.copy()


        df_cb_uns_preQ_SQ_1_log_[['beta_par_core', 'alph_c','alph_b']] = \
        np.log10(df_cb_uns_preQ_SQ_1_log_[['beta_par_core', 'alph_c','alph_b']])
        
        df_cb_uns_preQ_SQ_1_log_[['Pow_core', 'Pow_beam', 'kB_angle']] = \
        xgbr_c1_b1.predict(df_cb_uns_preQ_SQ_1_log_).astype(float)

        df_cb_uns_preQ_SQ_1_log_[['beta_par_core', 'alph_c','alph_b', 'Pow_core', 'Pow_beam', 'kB_angle']] = \
        10**df_cb_uns_preQ_SQ_1_log_[['beta_par_core', 'alph_c','alph_b', 'Pow_core', 'Pow_beam', 'kB_angle']]

        df_cb_uns_postQ_SQ_1_ = df_cb_uns_preQ_SQ_1_log_.copy()
        df_cb_uns_postQ_SQ_1_ = df_cb_uns_postQ_SQ_1_[['beta_par_core', 'alph_c', 'tau_b', 'D_b', 'alph_b', 'vv_b', 'Pow_core', 'Pow_beam', 'kB_angle']]
    else:
        df_cb_uns_postQ_SQ_1 = pd.DataFrame(columns = ['beta_par_core', 'alph_c', 'tau_b', 'D_b', 'alph_b', 'vv_b', 'Pow_core', 'Pow_beam', 'kB_angle'])

    if df_cb_uns_preQ_SQ_2_.size > 0:
        df_cb_uns_preQ_SQ_2_log_ = df_cb_uns_preQ_SQ_2_.copy()


        df_cb_uns_preQ_SQ_2_log_[['beta_par_core', 'alph_c','alph_b']] = \
        np.log10(df_cb_uns_preQ_SQ_2_log_[['beta_par_core', 'alph_c','alph_b']])
        
        df_cb_uns_preQ_SQ_2_log_[['Pow_core', 'kB_angle']] = \
        xgbr_c1_b0.predict(df_cb_uns_preQ_SQ_2_log_).astype(float)

        df_cb_uns_preQ_SQ_2_log_[['beta_par_core', 'alph_c','alph_b', 'Pow_core', 'kB_angle']] = \
        10**df_cb_uns_preQ_SQ_2_log_[['beta_par_core', 'alph_c','alph_b', 'Pow_core', 'kB_angle']]

        df_cb_uns_postQ_SQ_2_ = df_cb_uns_preQ_SQ_2_log_.copy()
        df_cb_uns_postQ_SQ_2_['Pow_beam'] = 0
        df_cb_uns_postQ_SQ_2_ = df_cb_uns_postQ_SQ_2_[['beta_par_core', 'alph_c', 'tau_b', 'D_b', 'alph_b', 'vv_b', 'Pow_core', 'Pow_beam', 'kB_angle']]
    else:
        df_cb_uns_postQ_SQ_2_ = pd.DataFrame(columns = ['beta_par_core', 'alph_c', 'tau_b', 'D_b', 'alph_b', 'vv_b', 'Pow_core', 'Pow_beam', 'kB_angle'])

    if df_cb_uns_preQ_SQ_3_.size > 0:
        df_cb_uns_preQ_SQ_3_log_ = df_cb_uns_preQ_SQ_3_.copy()


        df_cb_uns_preQ_SQ_3_log_[['beta_par_core', 'alph_c','alph_b']] = \
        np.log10(df_cb_uns_preQ_SQ_3_log_[['beta_par_core', 'alph_c','alph_b']])
        
        df_cb_uns_preQ_SQ_3_log_[['Pow_core', 'kB_angle']] = \
        xgbr_c1_b0.predict(df_cb_uns_preQ_SQ_3_log_).astype(float)

        df_cb_uns_preQ_SQ_3_log_[['beta_par_core', 'alph_c','alph_b', 'Pow_core', 'kB_angle']] = \
        10**df_cb_uns_preQ_SQ_3_log_[['beta_par_core', 'alph_c','alph_b', 'Pow_core', 'kB_angle']]

        df_cb_uns_postQ_SQ_3_ = df_cb_uns_preQ_SQ_3_log_.copy()
        df_cb_uns_postQ_SQ_3_['Pow_beam'] = 0
        df_cb_uns_postQ_SQ_3_ = df_cb_uns_postQ_SQ_3_[['beta_par_core', 'alph_c', 'tau_b', 'D_b', 'alph_b', 'vv_b', 'Pow_core', 'Pow_beam', 'kB_angle']]
    else:
        df_cb_uns_postQ_SQ_3_ = pd.DataFrame(columns = ['beta_par_core', 'alph_c', 'tau_b', 'D_b', 'alph_b', 'vv_b', 'Pow_core', 'Pow_beam', 'kB_angle'])

    if df_cb_uns_preQ_SQ_4_.size > 0:
        df_cb_uns_preQ_SQ_4_log_ = df_cb_uns_preQ_SQ_4_.copy()


        df_cb_uns_preQ_SQ_4_log_[['beta_par_core', 'alph_c','alph_b']] = \
        np.log10(df_cb_uns_preQ_SQ_4_log_[['beta_par_core', 'alph_c','alph_b']])
        
        df_cb_uns_preQ_SQ_4_log_[['Pow_beam', 'kB_angle']] = \
        xgbr_c0_b1_k1.predict(df_cb_uns_preQ_SQ_4_log_).astype(float)

        df_cb_uns_preQ_SQ_4_log_[['beta_par_core', 'alph_c','alph_b', 'Pow_beam', 'kB_angle']] = \
        10**df_cb_uns_preQ_SQ_4_log_[['beta_par_core', 'alph_c','alph_b', 'Pow_beam', 'kB_angle']]

        df_cb_uns_postQ_SQ_4_ = df_cb_uns_preQ_SQ_4_log_.copy()
        df_cb_uns_postQ_SQ_4_['Pow_core'] = 0
        df_cb_uns_postQ_SQ_4_ = df_cb_uns_postQ_SQ_4_[['beta_par_core', 'alph_c', 'tau_b', 'D_b', 'alph_b', 'vv_b', 'Pow_core', 'Pow_beam', 'kB_angle']]
    else:
        df_cb_uns_postQ_SQ_4_ = pd.DataFrame(columns = ['beta_par_core', 'alph_c', 'tau_b', 'D_b', 'alph_b', 'vv_b', 'Pow_core', 'Pow_beam', 'kB_angle'])

    if df_cb_uns_preQ_SQ_5_.size > 0:
        df_cb_uns_preQ_SQ_5_log_ = df_cb_uns_preQ_SQ_5_.copy()


        df_cb_uns_preQ_SQ_5_log_[['beta_par_core', 'alph_c','alph_b']] = \
        np.log10(df_cb_uns_preQ_SQ_5_log_[['beta_par_core', 'alph_c','alph_b']])
        
        df_cb_uns_preQ_SQ_5_log_[['Pow_beam', 'kB_angle']] = \
        xgbr_c0_b1_k0.predict(df_cb_uns_preQ_SQ_5_log_).astype(float)

        df_cb_uns_preQ_SQ_5_log_[['beta_par_core', 'alph_c','alph_b', 'Pow_beam', 'kB_angle']] = \
        10**df_cb_uns_preQ_SQ_5_log_[['beta_par_core', 'alph_c','alph_b', 'Pow_beam', 'kB_angle']]

        df_cb_uns_postQ_SQ_5_ = df_cb_uns_preQ_SQ_5_log_.copy()
        df_cb_uns_postQ_SQ_5_['Pow_core'] = 0
        df_cb_uns_postQ_SQ_5_ = df_cb_uns_postQ_SQ_5_[['beta_par_core', 'alph_c', 'tau_b', 'D_b', 'alph_b', 'vv_b', 'Pow_core', 'Pow_beam', 'kB_angle']]
    else:
        df_cb_uns_postQ_SQ_5_ = pd.DataFrame(columns = ['beta_par_core', 'alph_c', 'tau_b', 'D_b', 'alph_b', 'vv_b', 'Pow_core', 'Pow_beam', 'kB_angle'])

    df_cb_uns_postQ_SQ_ = pd.concat([df_cb_uns_postQ_SQ_0_, df_cb_uns_postQ_SQ_1_, df_cb_uns_postQ_SQ_2_, \
                                 df_cb_uns_postQ_SQ_3_, df_cb_uns_postQ_SQ_4_, df_cb_uns_postQ_SQ_5_]).sort_index()
    
    df_cb_postQ_SQ_ = df_cb_.copy()

    df_cb_postQ_SQ_['group'] = np.nan
    df_cb_postQ_SQ_.loc[df_cb_uns_preQ_SQ_.index, 'group' ] = \
    df_cb_uns_preQ_SQ_['group']
    df_cb_postQ_SQ_['group'] = df_cb_postQ_SQ_['group']#.astype('Int64')

    df_cb_postQ_SQ_[['Pow_core', 'Pow_beam', 'kB_angle']] = np.nan
    df_cb_postQ_SQ_.loc[df_cb_uns_postQ_SQ_.index, ['Pow_core', 'Pow_beam', 'kB_angle'] ] = \
    df_cb_uns_postQ_SQ_[['Pow_core', 'Pow_beam', 'kB_angle'] ]
    
    return df_cb_postQ_SQ_

### run SAVIC-Q - CB 

In [7]:
df_cb_post_SQ = SAVIC_Q_CB(pd.read_hdf('00_SAVIC_Examples/SAVIC_Examples.h5', key = 'SAVIC-P_CB_post_P'))
df_cb_post_SQ

,beta_par_core,alph_c,tau_b,alph_b,D_b,vv_b,unstable,group,Pow_core,Pow_beam,kB_angle
0,1.0,1.0,1.0,1.0,0.05,0.5,False,NaN,NaN,NaN,NaN
1,1.5,2.5,0.8,1.0,0.05,0.5,True,3.0,0.193271,0.000000,0.004137
2,0.5,1.0,1.0,3.5,0.10,1.5,True,5.0,0.000000,0.123028,0.003983
3,0.8,1.1,1.0,1.2,0.05,1.8,True,5.0,0.000000,0.004923,0.001414
4,0.5,0.7,0.8,0.8,0.01,0.2,False,NaN,NaN,NaN,NaN
5,0.8,3.1,1.0,3.9,0.10,1.9,True,1.0,0.214777,0.000728,0.000991


## SAVIC-Q - core \& $\alpha$

### load SAVIC-Q - C$\alpha$ XGB classifier \& regressor

In [8]:
xgbc_kca = xgb.XGBClassifier()
xgbc_kca.load_model('Output/ML/models/xgbc_kca.json')

xgbr_c1_a1_k1 = xgb.XGBRegressor()
xgbr_c1_a1_k1.load_model('Output/ML/models/xgbr_ca_c1_a1_k1.json')

xgbr_c1_a1_k0 = xgb.XGBRegressor()
xgbr_c1_a1_k0.load_model('Output/ML/models/xgbr_ca_c1_a1_k0.json')

xgbr_c1_a0 = xgb.XGBRegressor()
xgbr_c1_a0.load_model('Output/ML/models/xgbr_ca_c1_a0_k0.json')

xgbr_c0_a1 = xgb.XGBRegressor()
xgbr_c0_a1.load_model('Output/ML/models/xgbr_ca_c0_a1_k0.json')

### load SAVIC-Q - C$\alpha$ processing function

In [9]:
def SAVIC_Q_CA(df_ca_):
    
    df_ca_uns_ = df_ca_[df_ca_['unstable']].drop(columns = ['unstable'])
    
    df_ca_uns_pre_SQ_ = df_ca_uns_.copy()
    df_ca_uns_pre_SQ_[['beta_par_core', 'alph_c', 'alph_a']] = \
    np.log10(df_ca_uns_pre_SQ_[['beta_par_core', 'alph_c', 'alph_a']])
    df_ca_uns_pre_SQ_.columns = [['log_beta_par_core', 'log_alph_c', 'tau_a', 'log_alph_a', 'D_a', 'vv_a']]
    
    df_ca_uns_postC_SQ_ = df_ca_uns_pre_SQ_.copy()
    df_ca_uns_postC_SQ_['group'] = xgbc_kca.predict(df_ca_uns_pre_SQ_.values)
    
    df_ca_uns_preQ_SQ_ = df_ca_uns_postC_SQ_.copy()
    df_ca_uns_preQ_SQ_[['log_beta_par_core', 'log_alph_c', 'log_alph_a']] = \
    pow(10., df_ca_uns_preQ_SQ_[['log_beta_par_core', 'log_alph_c', 'log_alph_a']]) 
    df_ca_uns_preQ_SQ_.columns = ['beta_par_core', 'alph_c', 'tau_a', 'alph_a', 'D_a', 'vv_a', 'group']
    
    df_ca_uns_preQ_SQ_0_ = df_ca_uns_preQ_SQ_[df_ca_uns_preQ_SQ_['group'] == 0].drop(columns = ['group'])
    df_ca_uns_preQ_SQ_1_ = df_ca_uns_preQ_SQ_[df_ca_uns_preQ_SQ_['group'] == 1].drop(columns = ['group'])
    df_ca_uns_preQ_SQ_2_ = df_ca_uns_preQ_SQ_[df_ca_uns_preQ_SQ_['group'] == 2].drop(columns = ['group'])
    df_ca_uns_preQ_SQ_3_ = df_ca_uns_preQ_SQ_[df_ca_uns_preQ_SQ_['group'] == 3].drop(columns = ['group'])
    df_ca_uns_preQ_SQ_4_ = df_ca_uns_preQ_SQ_[df_ca_uns_preQ_SQ_['group'] == 4].drop(columns = ['group'])
    df_ca_uns_preQ_SQ_5_ = df_ca_uns_preQ_SQ_[df_ca_uns_preQ_SQ_['group'] == 5].drop(columns = ['group'])
    
    # regressors
    if df_ca_uns_preQ_SQ_0_.size > 0:
        df_ca_uns_preQ_SQ_0_log_ = df_ca_uns_preQ_SQ_0_.copy()


        df_ca_uns_preQ_SQ_0_log_[['beta_par_core', 'alph_c','alph_a']] = np.log10(df_ca_uns_preQ_SQ_0_log_[['beta_par_core', 'alph_c','alph_a']])
        df_ca_uns_preQ_SQ_0_log_[['Pow_core', 'Pow_alpha', 'kB_angle']] = xgbr_c1_a1_k1.predict(df_ca_uns_preQ_SQ_0_log_).astype(float)

        df_ca_uns_preQ_SQ_0_log_[['beta_par_core', 'alph_c','alph_a', 'Pow_core', 'Pow_alpha', 'kB_angle']] = \
        10**df_ca_uns_preQ_SQ_0_log_[['beta_par_core', 'alph_c','alph_a', 'Pow_core', 'Pow_alpha', 'kB_angle']]

        df_ca_uns_postQ_SQ_0_ = df_ca_uns_preQ_SQ_0_log_.copy()
        df_ca_uns_postQ_SQ_0_ = df_ca_uns_postQ_SQ_0_[['beta_par_core', 'alph_c', 'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_alpha', 'kB_angle']]
    else:
        df_ca_uns_postQ_SQ_0_ = pd.DataFrame(columns = ['beta_par_core', 'alph_c', 'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_alpha', 'kB_angle'])

    if df_ca_uns_preQ_SQ_1_.size > 0:
        df_ca_uns_preQ_SQ_1_log_ = df_ca_uns_preQ_SQ_1_.copy()


        df_ca_uns_preQ_SQ_1_log_[['beta_par_core', 'alph_c','alph_a']] = np.log10(df_ca_uns_preQ_SQ_1_log_[['beta_par_core', 'alph_c','alph_a']])
        df_ca_uns_preQ_SQ_1_log_[['Pow_core', 'Pow_alpha', 'kB_angle']] = xgbr_c1_a1_k0.predict(df_ca_uns_preQ_SQ_1_log_).astype(float)

        df_ca_uns_preQ_SQ_1_log_[['beta_par_core', 'alph_c','alph_a', 'Pow_core', 'Pow_alpha', 'kB_angle']] = \
        10**df_ca_uns_preQ_SQ_1_log_[['beta_par_core', 'alph_c','alph_a', 'Pow_core', 'Pow_alpha', 'kB_angle']]

        df_ca_uns_postQ_SQ_1_ = df_ca_uns_preQ_SQ_1_log_.copy()
        df_ca_uns_postQ_SQ_1_ = df_ca_uns_postQ_SQ_1_[['beta_par_core', 'alph_c', 'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_alpha', 'kB_angle']]
    else:
        df_ca_uns_postQ_SQ_1_ = pd.DataFrame(columns = ['beta_par_core', 'alph_c', 'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_alpha', 'kB_angle'])

    if df_ca_uns_preQ_SQ_2_.size > 0:
        df_ca_uns_preQ_SQ_2_log_ = df_ca_uns_preQ_SQ_2_.copy()

        df_ca_uns_preQ_SQ_2_log_[['beta_par_core', 'alph_c','alph_a']] = np.log10(df_ca_uns_preQ_SQ_2_log_[['beta_par_core', 'alph_c','alph_a']])
        df_ca_uns_preQ_SQ_2_log_[['Pow_core', 'kB_angle']] = xgbr_c1_a0.predict(df_ca_uns_preQ_SQ_2_log_).astype(float)

        df_ca_uns_preQ_SQ_2_log_[['beta_par_core', 'alph_c','alph_a', 'Pow_core', 'kB_angle']] = \
        10**df_ca_uns_preQ_SQ_2_log_[['beta_par_core', 'alph_c','alph_a', 'Pow_core', 'kB_angle']]

        df_ca_uns_postQ_SQ_2_ = df_ca_uns_preQ_SQ_2_log_.copy()
        df_ca_uns_postQ_SQ_2_['Pow_alpha'] = 0
        df_ca_uns_postQ_SQ_2_ = df_ca_uns_postQ_SQ_2_[['beta_par_core', 'alph_c', 'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_alpha', 'kB_angle']]
    else:
        df_ca_uns_postQ_SQ_2_ = pd.DataFrame(columns = ['beta_par_core', 'alph_c', 'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_alpha', 'kB_angle'])

    if df_ca_uns_preQ_SQ_3_.size > 0:
        df_ca_uns_preQ_SQ_3_log_ = df_ca_uns_preQ_SQ_3_.copy()


        df_ca_uns_preQ_SQ_3_log_[['beta_par_core', 'alph_c','alph_a']] = np.log10(df_ca_uns_preQ_SQ_3_log_[['beta_par_core', 'alph_c','alph_a']])
        df_ca_uns_preQ_SQ_3_log_[['Pow_core', 'kB_angle']] = xgbr_c1_a0.predict(df_ca_uns_preQ_SQ_3_log_).astype(float)

        df_ca_uns_preQ_SQ_3_log_[['beta_par_core', 'alph_c','alph_a', 'Pow_core', 'kB_angle']] = \
        10**df_ca_uns_preQ_SQ_3_log_[['beta_par_core', 'alph_c','alph_a', 'Pow_core', 'kB_angle']]

        df_ca_uns_postQ_SQ_3_ = df_ca_uns_preQ_SQ_3_log_.copy()
        df_ca_uns_postQ_SQ_3_['Pow_alpha'] = 0
        df_ca_uns_postQ_SQ_3_ = df_ca_uns_postQ_SQ_3_[['beta_par_core', 'alph_c', 'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_alpha', 'kB_angle']]
    else:
        df_ca_uns_postQ_SQ_3_ = pd.DataFrame(columns = ['beta_par_core', 'alph_c', 'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_alpha', 'kB_angle'])

    if df_ca_uns_preQ_SQ_4_.size > 0:
        df_ca_uns_preQ_SQ_4_log_ = df_ca_uns_preQ_SQ_4_.copy()


        df_ca_uns_preQ_SQ_4_log_[['beta_par_core', 'alph_c','alph_a']] = np.log10(df_ca_uns_preQ_SQ_4_log_[['beta_par_core', 'alph_c','alph_a']])
        df_ca_uns_preQ_SQ_4_log_[['Pow_alpha', 'kB_angle']] = xgbr_c0_a1.predict(df_ca_uns_preQ_SQ_4_log_).astype(float)

        df_ca_uns_preQ_SQ_4_log_[['beta_par_core', 'alph_c','alph_a', 'Pow_alpha', 'kB_angle']] = \
        10**df_ca_uns_preQ_SQ_4_log_[['beta_par_core', 'alph_c','alph_a', 'Pow_alpha', 'kB_angle']]

        df_ca_uns_postQ_SQ_4_ = df_ca_uns_preQ_SQ_4_log_.copy()
        df_ca_uns_postQ_SQ_4_['Pow_core'] = 0
        df_ca_uns_postQ_SQ_4_ = df_ca_uns_postQ_SQ_4_[['beta_par_core', 'alph_c', 'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_alpha', 'kB_angle']]
    else:
        df_ca_uns_postQ_SQ_4_ = pd.DataFrame(columns = ['beta_par_core', 'alph_c', 'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_alpha', 'kB_angle'])

    if df_ca_uns_preQ_SQ_5_.size > 0:
        df_ca_uns_preQ_SQ_5_log_ = df_ca_uns_preQ_SQ_5_.copy()


        df_ca_uns_preQ_SQ_5_log_[['beta_par_core', 'alph_c','alph_a']] = np.log10(df_ca_uns_preQ_SQ_5_log_[['beta_par_core', 'alph_c','alph_a']])
        df_ca_uns_preQ_SQ_5_log_[['Pow_alpha', 'kB_angle']] = xgbr_c0_a1.predict(df_ca_uns_preQ_SQ_5_log_).astype(float)

        df_ca_uns_preQ_SQ_5_log_[['beta_par_core', 'alph_c','alph_a', 'Pow_alpha', 'kB_angle']] = \
        10**df_ca_uns_preQ_SQ_5_log_[['beta_par_core', 'alph_c','alph_a', 'Pow_alpha', 'kB_angle']]

        df_ca_uns_postQ_SQ_5_ = df_ca_uns_preQ_SQ_5_log_.copy()
        df_ca_uns_postQ_SQ_5_['Pow_core'] = 0
        df_ca_uns_postQ_SQ_5_ = df_ca_uns_postQ_SQ_5_[['beta_par_core', 'alph_c', 'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_alpha', 'kB_angle']]
    else:
        df_ca_uns_postQ_SQ_5_ = pd.DataFrame(columns = ['beta_par_core', 'alph_c', 'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_alpha', 'kB_angle'])

    df_ca_uns_postQ_SQ_ = pd.concat([df_ca_uns_postQ_SQ_0_, df_ca_uns_postQ_SQ_1_, df_ca_uns_postQ_SQ_2_, \
                                 df_ca_uns_postQ_SQ_3_, df_ca_uns_postQ_SQ_4_, df_ca_uns_postQ_SQ_5_]).sort_index()
    
    df_ca_postQ_SQ_ = df_ca_.copy()

    df_ca_postQ_SQ_['group'] = np.nan
    df_ca_postQ_SQ_.loc[df_ca_uns_preQ_SQ_.index, 'group' ] = \
    df_ca_uns_preQ_SQ_['group']
    df_ca_postQ_SQ_['group'] = df_ca_postQ_SQ_['group']#.astype('Int64')

    df_ca_postQ_SQ_[['Pow_core', 'Pow_alpha', 'kB_angle']] = np.nan
    df_ca_postQ_SQ_.loc[df_ca_uns_postQ_SQ_.index, ['Pow_core', 'Pow_alpha', 'kB_angle'] ] = \
    df_ca_uns_postQ_SQ_[['Pow_core', 'Pow_alpha', 'kB_angle'] ]
    
    return df_ca_postQ_SQ_

### run SAVIC-Q - C$\alpha$ 

In [10]:
df_ca_post_SQ = SAVIC_Q_CA(pd.read_hdf('00_SAVIC_Examples/SAVIC_Examples.h5', key = 'SAVIC-P_CA_post_P'))
df_ca_post_SQ

,beta_par_core,alph_c,tau_a,alph_a,D_a,vv_a,unstable,group,Pow_core,Pow_alpha,kB_angle
0,1.7,0.5,0.5,0.7,0.02,0.07,True,3.0,0.004490,0.000000,0.001962
1,0.6,1.0,0.4,1.9,0.03,0.06,True,5.0,0.000000,0.007504,0.004290
2,0.5,0.7,0.8,0.8,0.01,0.00,False,NaN,NaN,NaN,NaN
3,3.6,0.4,0.6,1.0,0.02,0.15,True,2.0,0.346296,0.000000,0.002381
4,1.0,1.0,0.5,1.2,0.09,0.20,False,NaN,NaN,NaN,NaN
5,0.2,2.4,0.2,0.9,0.03,0.15,True,3.0,0.030773,0.000000,0.002031


## SAVIC-Q - core, beam \& $\alpha$

### load SAVIC-Q - CB$\alpha$ XGB classifier \& regressor

In [11]:
xgbc_kcba = xgb.XGBClassifier()
xgbc_kcba.load_model('Output/ML/models/xgbc_kcba.json')

xgbr_c1_b1_a1 = xgb.XGBRegressor()
xgbr_c1_b1_a1.load_model('Output/ML/models/xgbr_cba_c1_b1_a1.json')

xgbr_c1_b1_a0 = xgb.XGBRegressor()
xgbr_c1_b1_a0.load_model('Output/ML/models/xgbr_cba_c1_b1_a0.json')

xgbr_c1_b0_a1 = xgb.XGBRegressor()
xgbr_c1_b0_a1.load_model('Output/ML/models/xgbr_cba_c1_b0_a1.json')

xgbr_c1_b0_a0 = xgb.XGBRegressor()
xgbr_c1_b0_a0.load_model('Output/ML/models/xgbr_cba_c1_b0_a0.json')

xgbr_c0_b1_a1 = xgb.XGBRegressor()
xgbr_c0_b1_a1.load_model('Output/ML/models/xgbr_cba_c0_b1_a1.json')

xgbr_c0_b1_a0_k1 = xgb.XGBRegressor()
xgbr_c0_b1_a0_k1.load_model('Output/ML/models/xgbr_cba_c0_b1_a0_k1.json')

xgbr_c0_b1_a0_k0 = xgb.XGBRegressor()
xgbr_c0_b1_a0_k0.load_model('Output/ML/models/xgbr_cba_c0_b1_a0_k0.json')

xgbr_c0_b0_a1 = xgb.XGBRegressor()
xgbr_c0_b0_a1.load_model('Output/ML/models/xgbr_cba_c0_b0_a1.json')



### load SAVIC-Q - CB$\alpha$ processing function

In [12]:
def SAVIC_Q_CBA(df_cba_):
    
    df_cba_uns_ = df_cba_[df_cba_['unstable']].drop(columns = ['unstable'])
    
    df_cba_uns_pre_SQ_ = df_cba_uns_.copy()
    df_cba_uns_pre_SQ_[['beta_par_core', 'alph_c', 'alph_b', 'alph_a']] = \
    np.log10(df_cba_uns_pre_SQ_[['beta_par_core', 'alph_c', 'alph_b', 'alph_a']])
    df_cba_uns_pre_SQ_.columns = [['log_beta_par_core', 'log_alph_c', \
                                   'tau_b', 'log_alph_b', 'D_b', 'vv_b', \
                                   'tau_a', 'log_alph_a', 'D_a', 'vv_a']]
    
    df_cba_uns_postC_SQ_ = df_cba_uns_pre_SQ_.copy()
    df_cba_uns_postC_SQ_['group'] = xgbc_kcba.predict(df_cba_uns_pre_SQ_.values)
    
    df_cba_uns_preQ_SQ_ = df_cba_uns_postC_SQ_.copy()
    df_cba_uns_preQ_SQ_[['log_beta_par_core', 'log_alph_c', 'log_alph_b', 'log_alph_a']] = \
    pow(10., df_cba_uns_preQ_SQ_[['log_beta_par_core', 'log_alph_c', 'log_alph_b', 'log_alph_a']]) 
    df_cba_uns_preQ_SQ_.columns = ['beta_par_core', 'alph_c', \
                                   'tau_b', 'alph_b', 'D_b', 'vv_b', \
                                   'tau_a', 'alph_a', 'D_a', 'vv_a', 'group']

    
    df_cba_uns_preQ_SQ_0_ = df_cba_uns_preQ_SQ_[df_cba_uns_preQ_SQ_['group'] == 0].drop(columns = ['group'])
    df_cba_uns_preQ_SQ_1_ = df_cba_uns_preQ_SQ_[df_cba_uns_preQ_SQ_['group'] == 1].drop(columns = ['group'])
    df_cba_uns_preQ_SQ_2_ = df_cba_uns_preQ_SQ_[df_cba_uns_preQ_SQ_['group'] == 2].drop(columns = ['group'])
    df_cba_uns_preQ_SQ_3_ = df_cba_uns_preQ_SQ_[df_cba_uns_preQ_SQ_['group'] == 3].drop(columns = ['group'])
    df_cba_uns_preQ_SQ_4_ = df_cba_uns_preQ_SQ_[df_cba_uns_preQ_SQ_['group'] == 4].drop(columns = ['group'])
    df_cba_uns_preQ_SQ_5_ = df_cba_uns_preQ_SQ_[df_cba_uns_preQ_SQ_['group'] == 5].drop(columns = ['group'])
    df_cba_uns_preQ_SQ_6_ = df_cba_uns_preQ_SQ_[df_cba_uns_preQ_SQ_['group'] == 6].drop(columns = ['group'])
    df_cba_uns_preQ_SQ_7_ = df_cba_uns_preQ_SQ_[df_cba_uns_preQ_SQ_['group'] == 7].drop(columns = ['group'])
    
    
    # regressors
    cba_full_col_list_ = ['beta_par_core', 'alph_c', \
                          'tau_b', 'D_b', 'alph_b', 'vv_b', \
                          'tau_a', 'D_a', 'alph_a', 'vv_a', \
                          'Pow_core', 'Pow_beam', 'Pow_alpha', 'kB_angle']

    # group 0 - C1B1A1
    if df_cba_uns_preQ_SQ_0_.size > 0:
        df_cba_uns_preQ_SQ_0_log_ = df_cba_uns_preQ_SQ_0_.copy()

        df_cba_uns_preQ_SQ_0_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a']] = \
        np.log10(df_cba_uns_preQ_SQ_0_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a']])
        df_cba_uns_preQ_SQ_0_log_[['Pow_core', 'Pow_beam', 'Pow_alpha', 'kB_angle']] = \
        xgbr_c1_b1_a1.predict(df_cba_uns_preQ_SQ_0_log_).astype(float)

        df_cba_uns_preQ_SQ_0_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_core', 'Pow_beam', 'Pow_alpha', 'kB_angle']] = \
        10**df_cba_uns_preQ_SQ_0_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_core', 'Pow_beam', 'Pow_alpha', 'kB_angle']]

        df_cba_uns_postQ_SQ_0_ = df_cba_uns_preQ_SQ_0_log_.copy()
        df_cba_uns_postQ_SQ_0_ = df_cba_uns_postQ_SQ_0_[cba_full_col_list_]
    else:
        df_cba_uns_postQ_SQ_0_ = pd.DataFrame(columns = cba_full_col_list_)



    # group 1 - C1B1A0
    if df_cba_uns_preQ_SQ_1_.size > 0:
        df_cba_uns_preQ_SQ_1_log_ = df_cba_uns_preQ_SQ_1_.copy()

        df_cba_uns_preQ_SQ_1_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a']] = \
        np.log10(df_cba_uns_preQ_SQ_1_log_[['beta_par_core', 'alph_c', 'alph_b', 'alph_a']])

        df_cba_uns_preQ_SQ_1_log_[['Pow_core', 'Pow_beam', 'kB_angle']] = \
        xgbr_c1_b1_a0.predict(df_cba_uns_preQ_SQ_1_log_).astype(float)

        df_cba_uns_preQ_SQ_1_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_core', 'Pow_beam', 'kB_angle']] = \
        10**df_cba_uns_preQ_SQ_1_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_core', 'Pow_beam', 'kB_angle']]

        df_cba_uns_postQ_SQ_1_ = df_cba_uns_preQ_SQ_1_log_.copy()
        df_cba_uns_postQ_SQ_1_['Pow_alpha'] = 0
        df_cba_uns_postQ_SQ_1_ = df_cba_uns_postQ_SQ_1_[cba_full_col_list_]
    else:
        df_cba_uns_postQ_SQ_1_ = pd.DataFrame(columns = cba_full_col_list_)

    # group 2 - C1B0A1
    if df_cba_uns_preQ_SQ_2_.size > 0:
        df_cba_uns_preQ_SQ_2_log_ = df_cba_uns_preQ_SQ_2_.copy()

        df_cba_uns_preQ_SQ_2_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a']] = \
        np.log10(df_cba_uns_preQ_SQ_2_log_[['beta_par_core', 'alph_c', 'alph_b', 'alph_a']])

        df_cba_uns_preQ_SQ_2_log_[['Pow_core', 'Pow_alpha', 'kB_angle']] = \
        xgbr_c1_b0_a1.predict(df_cba_uns_preQ_SQ_2_log_).astype(float)

        df_cba_uns_preQ_SQ_2_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_core', 'Pow_alpha', 'kB_angle']] = \
        10**df_cba_uns_preQ_SQ_2_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_core', 'Pow_alpha', 'kB_angle']]

        df_cba_uns_postQ_SQ_2_ = df_cba_uns_preQ_SQ_2_log_.copy()
        df_cba_uns_postQ_SQ_2_['Pow_beam'] = 0
        df_cba_uns_postQ_SQ_2_ = df_cba_uns_postQ_SQ_2_[cba_full_col_list_]
    else:
        df_cba_uns_postQ_SQ_2_ = pd.DataFrame(columns = cba_full_col_list_)

    # group 3 - C1B0A0
    if df_cba_uns_preQ_SQ_3_.size > 0:
        df_cba_uns_preQ_SQ_3_log_ = df_cba_uns_preQ_SQ_3_.copy()

        df_cba_uns_preQ_SQ_3_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a']] = \
        np.log10(df_cba_uns_preQ_SQ_3_log_[['beta_par_core', 'alph_c', 'alph_b', 'alph_a']])

        df_cba_uns_preQ_SQ_3_log_[['Pow_core', 'kB_angle']] = \
        xgbr_c1_b0_a0.predict(df_cba_uns_preQ_SQ_3_log_).astype(float)

        df_cba_uns_preQ_SQ_3_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_core', 'kB_angle']] = \
        10**df_cba_uns_preQ_SQ_3_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_core', 'kB_angle']]

        df_cba_uns_postQ_SQ_3_ = df_cba_uns_preQ_SQ_3_log_.copy()
        df_cba_uns_postQ_SQ_3_[['Pow_beam', 'Pow_alpha']] = 0
        df_cba_uns_postQ_SQ_3_ = df_cba_uns_postQ_SQ_3_[cba_full_col_list_]
    else:
        df_cba_uns_postQ_SQ_3_ = pd.DataFrame(columns = cba_full_col_list_)

    # group 4 - C0B1A1
    if df_cba_uns_preQ_SQ_4_.size > 0:
        df_cba_uns_preQ_SQ_4_log_ = df_cba_uns_preQ_SQ_4_.copy()

        df_cba_uns_preQ_SQ_4_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a']] = \
        np.log10(df_cba_uns_preQ_SQ_4_log_[['beta_par_core', 'alph_c', 'alph_b', 'alph_a']])

        df_cba_uns_preQ_SQ_4_log_[['Pow_beam', 'Pow_alpha', 'kB_angle']] = \
        xgbr_c0_b1_a1.predict(df_cba_uns_preQ_SQ_4_log_).astype(float)

        df_cba_uns_preQ_SQ_4_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_beam', 'Pow_alpha', 'kB_angle']] = \
        10**df_cba_uns_preQ_SQ_4_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_beam', 'Pow_alpha', 'kB_angle']]

        df_cba_uns_postQ_SQ_4_ = df_cba_uns_preQ_SQ_4_log_.copy()
        df_cba_uns_postQ_SQ_4_[['Pow_core']] = 0
        df_cba_uns_postQ_SQ_4_ = df_cba_uns_postQ_SQ_4_[cba_full_col_list_]
    else:
        df_cba_uns_postQ_SQ_4_ = pd.DataFrame(columns = cba_full_col_list_)


    # group 5 - C0B1A0k1
    if df_cba_uns_preQ_SQ_5_.size > 0:
        df_cba_uns_preQ_SQ_5_log_ = df_cba_uns_preQ_SQ_5_.copy()

        df_cba_uns_preQ_SQ_5_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a']] = \
        np.log10(df_cba_uns_preQ_SQ_5_log_[['beta_par_core', 'alph_c', 'alph_b', 'alph_a']])

        df_cba_uns_preQ_SQ_5_log_[['Pow_beam', 'kB_angle']] = \
        xgbr_c0_b1_a0_k1.predict(df_cba_uns_preQ_SQ_5_log_).astype(float)

        df_cba_uns_preQ_SQ_5_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_beam', 'kB_angle']] = \
        10**df_cba_uns_preQ_SQ_5_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_beam', 'kB_angle']]

        df_cba_uns_postQ_SQ_5_ = df_cba_uns_preQ_SQ_5_log_.copy()
        df_cba_uns_postQ_SQ_5_[['Pow_core', 'Pow_alpha']] = 0
        df_cba_uns_postQ_SQ_5_ = df_cba_uns_postQ_SQ_5_[cba_full_col_list_]
    else:
        df_cba_uns_postQ_SQ_5_ = pd.DataFrame(columns = cba_full_col_list_)    

    # group 6 - C0B1A0k0
    if df_cba_uns_preQ_SQ_6_.size > 0:
        df_cba_uns_preQ_SQ_6_log_ = df_cba_uns_preQ_SQ_6_.copy()

        df_cba_uns_preQ_SQ_6_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a']] = \
        np.log10(df_cba_uns_preQ_SQ_6_log_[['beta_par_core', 'alph_c', 'alph_b', 'alph_a']])

        df_cba_uns_preQ_SQ_6_log_[['Pow_beam', 'kB_angle']] = \
        xgbr_c0_b1_a0_k0.predict(df_cba_uns_preQ_SQ_6_log_).astype(float)

        df_cba_uns_preQ_SQ_6_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_beam', 'kB_angle']] = \
        10**df_cba_uns_preQ_SQ_6_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_beam', 'kB_angle']]

        df_cba_uns_postQ_SQ_6_ = df_cba_uns_preQ_SQ_6_log_.copy()
        df_cba_uns_postQ_SQ_6_[['Pow_core', 'Pow_alpha']] = 0
        df_cba_uns_postQ_SQ_6_ = df_cba_uns_postQ_SQ_6_[cba_full_col_list_]
    else:
        df_cba_uns_postQ_SQ_6_ = pd.DataFrame(columns = cba_full_col_list_)    


    # group 7 - C0B0A1
    if df_cba_uns_preQ_SQ_7_.size > 0:
        df_cba_uns_preQ_SQ_7_log_ = df_cba_uns_preQ_SQ_7_.copy()

        df_cba_uns_preQ_SQ_7_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a']] = \
        np.log10(df_cba_uns_preQ_SQ_7_log_[['beta_par_core', 'alph_c', 'alph_b', 'alph_a']])

        df_cba_uns_preQ_SQ_7_log_[['Pow_alpha', 'kB_angle']] = \
        xgbr_c0_b0_a1.predict(df_cba_uns_preQ_SQ_7_log_).astype(float)

        df_cba_uns_preQ_SQ_7_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_alpha', 'kB_angle']] = \
        10**df_cba_uns_preQ_SQ_7_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_alpha', 'kB_angle']]

        df_cba_uns_postQ_SQ_7_ = df_cba_uns_preQ_SQ_7_log_.copy()
        df_cba_uns_postQ_SQ_7_[['Pow_core', 'Pow_beam']] = 0
        df_cba_uns_postQ_SQ_7_ = df_cba_uns_postQ_SQ_7_[cba_full_col_list_]
    else:
        df_cba_uns_postQ_SQ_7_ = pd.DataFrame(columns = cba_full_col_list_)    


    df_cba_uns_postQ_SQ_ = pd.concat([df_cba_uns_postQ_SQ_0_, df_cba_uns_postQ_SQ_1_, df_cba_uns_postQ_SQ_2_, \
                                      df_cba_uns_postQ_SQ_3_, df_cba_uns_postQ_SQ_4_, df_cba_uns_postQ_SQ_5_, \
                                      df_cba_uns_postQ_SQ_6_,df_cba_uns_postQ_SQ_7_]).sort_index()
    
    df_cba_postQ_SQ_ = df_cba_.copy()

    df_cba_postQ_SQ_['group'] = np.nan
    df_cba_postQ_SQ_.loc[df_cba_uns_preQ_SQ_.index, 'group' ] = \
    df_cba_uns_preQ_SQ_['group']
    df_cba_postQ_SQ_['group'] = df_cba_postQ_SQ_['group']#.astype('Int64')

    df_cba_postQ_SQ_[['Pow_core', 'Pow_beam', 'Pow_alpha', 'kB_angle']] = np.nan
    df_cba_postQ_SQ_.loc[df_cba_uns_postQ_SQ_.index, ['Pow_core', 'Pow_beam', 'Pow_alpha', 'kB_angle'] ] = \
    df_cba_uns_postQ_SQ_[['Pow_core', 'Pow_beam', 'Pow_alpha', 'kB_angle'] ]
    
    return df_cba_postQ_SQ_

### run SAVIC-Q - CB$\alpha$ 

In [13]:
df_cba_post_SQ = SAVIC_Q_CBA(pd.read_hdf('00_SAVIC_Examples/SAVIC_Examples.h5', key = 'SAVIC-P_CBA_post_P'))
df_cba_post_SQ

,beta_par_core,alph_c,tau_b,alph_b,D_b,vv_b,tau_a,alph_a,D_a,vv_a,unstable,group,Pow_core,Pow_beam,Pow_alpha,kB_angle
0,1.7,0.5,1.0,1.0,0.05,0.5,0.5,0.7,0.02,0.07,True,3.0,0.004833,0.000000,0.000000,0.001945
1,0.6,1.0,0.8,1.0,0.05,0.5,0.4,1.9,0.03,0.06,True,7.0,0.000000,0.000000,0.005254,0.004523
2,0.5,0.7,1.0,3.5,0.10,1.5,0.8,0.8,0.01,0.00,True,6.0,0.000000,0.226850,0.000000,0.004767
3,2.2,0.8,1.0,0.7,0.10,0.9,0.6,1.0,0.02,0.15,False,NaN,NaN,NaN,NaN,NaN
4,3.6,0.4,1.2,0.7,0.10,0.8,0.6,1.0,0.03,0.13,True,1.0,0.008698,0.011312,0.000000,0.273413
5,1.0,1.0,1.0,1.2,0.05,1.8,0.5,1.2,0.09,0.20,True,6.0,0.000000,0.012197,0.000000,0.001371
6,0.2,2.4,1.0,3.1,0.10,1.5,0.2,0.9,0.03,0.15,True,3.0,0.035672,0.000000,0.000000,0.001954
